# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [17]:
city_csv = "../../WeatherPy/starter_code/output_data/cities.csv"
city_df = pd.read_csv(city_csv)

city_df = city_df[["City","Cloudiness","Country","Date", "Humidity",
       "Latitude", "Longitude", "Max Temp", "Wind Speed"]]
city_df

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,Dikson,99,RU,1612498876,88,73.5069,80.5464,-14.26,10.49
1,Hithadhoo,100,MV,1612498856,76,-0.6000,73.0833,82.54,11.52
2,George Town,20,MY,1612499022,51,5.4112,100.3354,87.80,2.30
3,Albany,18,US,1612498809,64,42.6001,-73.9662,26.01,2.82
4,Avarua,97,CK,1612498989,65,-21.2078,-159.7750,84.20,6.91
...,...,...,...,...,...,...,...,...,...
579,Whitianga,43,NZ,1612499112,58,-36.8333,175.7000,68.04,4.92
580,Troitskoye,100,RU,1612499112,94,52.9821,84.6757,14.02,2.62
581,Waddān,17,LY,1612499055,87,29.1614,16.1390,55.99,4.29
582,Rio Branco,75,BR,1612499112,100,-9.9747,-67.8100,75.20,4.61


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [21]:
gmaps.configure(api_key=g_key)

humidity_locations = city_df[["Latitude", "Longitude"]]
humidity_weight = city_df["Humidity"]

humidity_fig = gmaps.figure()
humidity_heat_layer = gmaps.heatmap_layer(humidity_locations, weights=humidity_weight, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
humidity_fig.add_layer(humidity_heat_layer)
humidity_fig

Figure(layout=FigureLayout(height='420px'))

In [22]:
city_df.count()

City          584
Cloudiness    584
Country       581
Date          584
Humidity      584
Latitude      584
Longitude     584
Max Temp      584
Wind Speed    584
dtype: int64

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
